# Import Libraries

In [15]:
using DataFrames
using DataFramesMeta
using PyCall
using Statistics
using PlotlyJS
using TimeSeries
using Random

@pyimport sklearn.metrics as Metrics
@pyimport sklearn.ensemble as Ensemble
@pyimport sklearn.tree as Tree
@pyimport sklearn.datasets as Datasets
@pyimport sklearn.model_selection as ModelSelection

include("GenerateSyntheticData.jl");
include("SingleFeatureImportance.jl");

# Generate Synthetic Test Data

In [16]:
X, y = getTestData(
    nFeatures=40,
    nInformative=5, 
    nRedundant=30,
    nSamples=500,
    sigmaStd=0.1);

# Fit SFI

In [17]:
classifier = Tree.DecisionTreeClassifier(
    criterion="entropy",
    max_features=1,
    class_weight="balanced",
    min_weight_fraction_leaf=0)

result = featureImportanceSFI(
    classifier,
    X, y,
    10,
    scoring="accuracy"
);

In [18]:
sort!(result, [:Mean], rev=true)


,FeatureName,Mean,StandardDeviation
,Any,Any,Any
1,I_1,0.57,0.0633333
2,R_17,0.566,0.0574302
3,R_19,0.556,0.0212498
4,R_2,0.55,0.0246306
5,R_1,0.548,0.0613333
6,R_6,0.548,0.0572092
7,R_10,0.544,0.0592396
8,R_20,0.544,0.0698443
9,R_14,0.522,0.0753333


# Plot Results

In [19]:
templates.default = "plotly_dark";
PlotlyJS.templates

toSavePlot = plot(
    bar(
        result,
        x=:Mean,
        y=:FeatureName,
        error_x=attr(type="data", array=:StandardDeviation, visible=true),
        orientation="h",
        ),
    PlotlyJS.Layout(
        title="SFI Results",
        width=800, height=1200,
        xaxis_title="Feature Name",
        yaxis_title="Feature Importance",
    )
)

# Save Results

In [ ]:
PlotlyJS.savefig(toSavePlot, "Figs/SFI_results.png")